In [ ]:
from appgeopy import *
from my_packages import *

In [ ]:
# ------------------------------------------------------------------------------
# Utility Functions
# ------------------------------------------------------------------------------
def timestamp_to_string(timestamp):
    """
    Convert a timestamp to a string formatted as 'NYYYYMMDD'.

    Parameters:
    - timestamp (pd.Timestamp): The timestamp to convert.

    Returns:
    - str: The formatted string.
    """
    return "N" + timestamp.strftime("%Y%m%d")


# ------------------------------------------------------------------------------
# Data Paths and Initial Setup
# ------------------------------------------------------------------------------
# Define the path to the Excel file containing LOS difference data
fpath = r"E:\030_CHOUSHUI_2024\000_INSCALDEFO_2_INSSTACKPSI\PROCESS_004\14_POST-PROCESSING\InSAR_GPS_LOS_Difference.xlsx"

# Get the available sheet names in the Excel file
available_sheetnames = data_io.get_sheetnames(fpath)

# ------------------------------------------------------------------------------
# Process Data from Excel Sheets
# ------------------------------------------------------------------------------
summary = pd.DataFrame(data=None, index=None)

# Iterate over each sheet name to compile data into a summary DataFrame
for select_station in tqdm(available_sheetnames, desc="Processing sheets"):
    df_by_station = pd.read_excel(
        fpath, sheet_name=select_station, parse_dates=[0], index_col=[0]
    )

    _temp = df_by_station.rename(columns={"LOS(mm)": select_station})
    summary = pd.concat([summary, _temp], axis=1)

# Rename the axis and columns for clarity
summary = summary.rename_axis("Datetime")
transposed_summary = summary.T
transposed_summary.columns = transposed_summary.columns.map(timestamp_to_string)
transposed_summary = transposed_summary.rename_axis("STATION")
# transposed_summary = transposed_summary.abs()  # Take absolute values for comparison

# ------------------------------------------------------------------------------
# Load GPS Shapefile Data
# ------------------------------------------------------------------------------
# Define the path to the GPS shapefile
gps_shp_fpath = r"D:\1000_SCRIPTS\002_PostQE_Scripts\20240527_GPS_Data\20240605\Shapefiles\2023_NewGPS_Locations_TWD97.shp"

# Read the GPS shapefile into a GeoDataFrame
gps_shp_geodf = gpd.read_file(gps_shp_fpath)

# Set the index of the GeoDataFrame to the 'STATION' column
gps_shp_geodf = gps_shp_geodf.set_index("STATION")

# ------------------------------------------------------------------------------
# Load Study Area boundary
# ------------------------------------------------------------------------------
study_shp_fpath = (
    r"E:\002_ARCGIS_WORK\ManuscriptArcGIS\choushui_modified_2021_TWD97.shp"
)
study_shp_geodf = gpd.read_file(study_shp_fpath)

# ------------------------------------------------------------------------------
# Merge and Prepare Data for Plotting
# ------------------------------------------------------------------------------
# List of columns in the transposed summary DataFrame
error_columns = transposed_summary.columns.tolist()

In [ ]:
# Select the first column for initial visualization
# select_col = "N20220712"
select_col = error_columns[0]

# Merge the GPS GeoDataFrame with the selected column from the transposed summary DataFrame
_temp = gps_shp_geodf.copy()
_temp[select_col] = _temp.index.map(transposed_summary[select_col])
_temp = _temp.dropna(how="any")

fig, ax = spatial_plot.spatial_interpolation(
    data=_temp,
    x_col="X_TWD97",
    y_col="Y_TWD97",
    value_col=select_col,
    title=select_col,
    method="inverse",
    cmap="turbo",
    vmin=-20,
    vmax=20,
    xlabel="",
    ylabel="",
    alpha=0.3,
    show_colorbar=True,
)

spatial_plot.point_values(
    gdf=_temp,
    value_column=select_col,
    vmin=-20,
    vmax=20,
    cmap="turbo",
    alpha=1,
    marker="^",
    s=75,
    edgecolors="black",
    linewidths=0.5,
    show_colorbar=False,
    ax=ax,
)

spatial_plot.show_polygons(
    gdf=study_shp_geodf, facecolor="none", alpha=1, lw=2, ax=ax
)

spatial_plot.show_anomalies(
    gdf=_temp,
    x_col="X_TWD97",
    y_col="Y_TWD97",
    value_col=select_col,
    ax=ax,
    label_offset=700,
)

ax.ticklabel_format(axis="both", style="sci", scilimits=[-5, 3])
spatial_plot.add_basemap(ax=ax, crs="EPSG:3826")

plt.show()